# DLite

## Test 1 - simple conversion

In [ ]:
# import pint

# ureg = pint.UnitRegistry()
# ureg.C

In [ ]:
import dlite

In [ ]:
from pathlib import Path

dlite.python_storage_plugin_path.append(Path("drivers").absolute())

In [ ]:
json_instance = dlite.Instance.from_location("CyclingStorageParser", "data/test.json")

In [ ]:
print(json_instance)

# CSV

In [ ]:
cycling_time_series = dlite.Instance.from_location(
    "csv",
    "data/test.csv",
    "mode=r;meta=https://example.com/meta/0.1/CyclingTimeSeries;infer=False",
)

In [ ]:
print(cycling_time_series)

## Test 2 - advanced conversion

### Ontology

- `CurrentTimeCurve`
  - Predicates:
    - `isA -> Curve`
    - `isA -> TimeSeries`
    - `hasProperty -> Time`
    - `hasProperty -> Current`
    - Predicates:
      - `isA -> MeasuredProperty`
- `ChargeTimeCurve`?
  - Predicates:
    - `isA -> Curve`
    - `isA -> TimeSeries`
    - `hasProperty -> Time`
    - `hasProperty -> Charge`
    - Predicates:
      - `isA -> MeasuredProperty`
      - `isDerivedFrom -> Time`?
      - `isDerivedFrom -> Current`?

In [ ]:
import numpy as np
from scipy.integrate import cumtrapz


def compute_capacities(t: np.ndarray, I: np.ndarray) -> np.ndarray:  # noqa: E741
    """Compute capacities as integral of current over time."""
    return cumtrapz(I, t, axis=0, initial=0)

In [ ]:
from pathlib import Path

from tripper import EMMO, RDFS, Triplestore

import dlite

# Paths
thisdir = Path(".").absolute()
datadir = thisdir / "data"
modeldir = thisdir / "models"
dlite.storage_path.append(f"{modeldir}/*.json")

# Create collection -- our knowledge base in this example
coll = dlite.Collection()

# Create a triplestore "view" of our knowledge base
ts = Triplestore(backend="collection", collection=coll)

# Add namespaces
DON = ts.bind("don", "http://example.com/demo-ontology#")
CyclingTimeSeries = ts.bind(
    "CyclingTimeSeries",
    "https://example.com/meta/0.1/CyclingTimeSeries#",
)
ChargeTimeSeries = ts.bind(
    "ChargeTimeSeries",
    "https://example.com/meta/0.1/ChargeTimeSeries#",
)

# Load data
cycling_time_series = dlite.Instance.from_location(
    "CyclingStorageParser",
    f"{datadir}/test.json",
)
coll.add("CyclingTimeSeries", cycling_time_series)


# 1. Map input datamodels -- data provider + ontologist
# -----------------------------------------------------

# Ontologist adds missing concepts
# ts.add_triples([])

# Add mappings for the input data models -- data provider
ts.add_mapsTo(DON.Time, CyclingTimeSeries.t)
ts.add_mapsTo(DON.Current, CyclingTimeSeries.I)


# 2. Map output datamodels -- modeller + ontologist
# -------------------------------------------------

# Ontologist adds missing concepts
# ts.add_triples([])

# Add mappings for the output data model -- modeller
ts.add_mapsTo(DON.Time, ChargeTimeSeries.t)
ts.add_mapsTo(DON.Charge, ChargeTimeSeries.Q)


# 3. Add mappings for conversion functions -- ontologist
# ------------------------------------------------------
ts.add_function(
    compute_capacities,
    expects=[DON.Time, DON.Current],
    returns=[DON.Charge],
    standard="fno",
)


# 4. Instantiate a capacity vs. cycle data instance -- modeller
# -------------------------------------------------------------
(charge_time_series,) = coll.get_instances(
    metaid=ChargeTimeSeries,
    property_mappings=True,
    function_repo=ts.function_repo,
)

print("charge time series instance:")
print(charge_time_series)